### https://medium.com/@achillesmoraites/fine-tuning-roberta-for-topic-classification-with-hugging-face-transformers-and-datasets-library-c6f8432d0820

In [1]:
!pip install transformers -U

In [2]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [3]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [4]:
notebook_login()

In [5]:
model_id = "roberta-base"
dataset_id = "amazon_polarity"
# relace the value with your model: ex <hugging-face-user>/<model-name>
repository_id = "AnonymousSub/Recipe_QA_CLIP_and_binary_ranking_style_1_epochs_recipe_triplet_recipes-roberta-base"

### Prepossessing

In [6]:
! pip install -U datasets

In [7]:
# Load dataset
dataset = load_dataset(dataset_id)

# Training and testing datasets
# train_dataset = dataset['train']
# test_dataset = dataset["test"].shard(num_shards=2, index=0)

# Validation dataset
# val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
# def tokenize(batch):
#     return tokenizer(batch["content"], padding=True, truncation=True, max_length=256)
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)

#tokenized_datasets = dataset.map(tokenize_function, batched=True) #保存了经过 tokenizer 处理后的数据集
#tokenized_datasets.save_to_disk("/Users/liang/Downloads/tokenized_datasets")
from datasets import load_from_disk

# 加载tokenized数据集
tokenized_datasets = load_from_disk("/Users/liang/Downloads/tokenized_datasets")

# train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
# val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
# test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
n=1500
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(n))  #shuffle(seed=42) 对数据集进行洗牌（随机化顺序
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(n))


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set the dataset format

In [9]:
#这段代码将经过 tokenization 处理后的数据集 tokenized_datasets 中的训练集和测试集进行了处理，选择了其中的前 n 个样本。
# n=1500
# train_dataset = train_dataset.shuffle(seed=42).select(range(n))  #shuffle(seed=42) 对数据集进行洗牌（随机化顺序
# val_dataset = val_dataset.shuffle(seed=42).select(range(n))
# test_dataset= test_dataset.shuffle(seed=42).select(range(n))

In [10]:
# Set dataset format
# train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
small_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
small_eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

### To make our model easier to use, we will create an id2label mapping. This will map the class ids to their labels.

This makes it easier to interpret the model’s output during inference.

In [11]:
# We will need this to directly output the class names when using the pipeline without mapping the labels later.
# Extract the number of classes and their names
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

number of labels: 2
the labels: ['negative', 'positive']


### Training and Evaluation
Now, we will set up our training parameters, Hugging Face 🤗 repository, and Tensorboard.

In [12]:
# ! pip install  accelerate -U

In [13]:
# !pip install huggingface_hub
# !python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('nika')"

In [14]:
!pip install evaluate

In [15]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")
def compute_metrices(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [16]:
import os

path = "/Users/liang/Downloads/tokenized_datasets"
if os.path.exists(path):
    print("資料集已儲存在指定的路徑。")
else:
    print("資料集尚未儲存。")


資料集已儲存在指定的路徑。


In [17]:
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)
# import os
# os.environ['HUGGINGFACE_TOKEN'] = 'dodobear'
# !pip install transformers -U
# ! pip install  accelerate -U
from transformers import (
    TrainingArguments,
    Trainer,)
! pip install  accelerate -U
# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    fp16=False,
    hub_token=HfFolder.get_token(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrices,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### We can start the training process by running

In [18]:
# Fine-tune the model
trainer.train()

# 儲存微調後的模型
model_save_path = "/Users/liang/Downloads/fine_tuned_model"
trainer.save_model(model_save_path)

# 載入已經儲存的模型
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained(model_save_path)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.413000,0.322548,0.906667


### Evaluate the model

In [19]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.3225482702255249,
 'eval_accuracy': 0.9066666666666666,
 'eval_runtime': 3114.6506,
 'eval_samples_per_second': 0.482,
 'eval_steps_per_second': 0.12,
 'epoch': 1.0}

### Publishing
We are ready to publish our model to Hugging Face

In [20]:
# # Save our tokenizer and create a model card
# tokenizer.save_pretrained(repository_id)
# trainer.create_model_card()
# # Push the results to the hub
# trainer.push_to_hub()

### Test the Model

In [21]:
# TEST MODEL

from transformers import pipeline

classifier = pipeline('text-classification',repository_id)

text = "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his innocence and vowing: quot;After the crucifixion comes the resurrection. quot; .."
text1='this product not bad'
result = classifier(text)

predicted_label = result[0]["label"]
print(f"Predicted label: {predicted_label}")

OSError: ignored